In [1]:
import requests
import json

def get_user_prompt(query: str) -> str:
    prompt = """
        User request: {query}

        # Example requests and responses:
        
        Input: "I need to see the win/loss report from last week"
        Output: {{
            "function_called": "/get_winlost_report"
        }}

        Input: "I want to get the reward report"
        Output: {{
            "function_called": "/get_reward_report"
        }}

        Input: "I want to take bet count report for user 123"
        Output: {{
            "function_called": "/get_betcount_report"
        }}

        Input: "Get me the get gross commission report for March transactions"
        Output: {{
            "function_called": "/get_gross_comm_report"
        }}

        Input: "I want get performance of abc1 last week"
        Output: {{
            "function_called": "N/A"
        }}
        
        Input: "Hello how are you today?"
        Output: {{
            "function_called": "N/A"
        }}
        
        Input: "I want Sportsbook only"
        Output: {{
            "function_called": "N/A"
        }}
        
        Based on this request, which function should be called? Return only the JSON response.
    """.format(query=query)
    
    return prompt


def functional_calling(query: str) -> str:
    # API endpoint
    url = "https://ollama.selab.edu.vn/api/chat"
    
    # Construct the system prompt using function metadata
    system_prompt = """
    You are an AI assistant that helps determine which function to call based on user input.
    Available functions:
        - /get_winlost_report: Get Win/Loss reports with filtering by date, product, and user level. Supports exporting reports (CSV/PDF), comparing trends across time periods, highlighting biggest winners/losers, and detecting unusual betting patterns.
        - /get_betcount_report: Get bet count reports showing total number of bets placed, filtered by date range, product, and user level. Includes bet frequency analysis, popular bet types, and peak betting times.
        - /get_turnover_report: Get turnover reports showing total betting volume/activity, with breakdowns by product, date range and user level. Includes trend analysis and comparison features.
        - /get_net_turnover_report: Get net turnover reports showing betting activity minus payouts/withdrawals. Provides profitability analysis by product, date range and user segments.
        - /get_gross_comm_report: Get gross commission reports showing total commission earned before deductions, filtered by date, product and agent level. Includes commission tier analysis.
        - /get_member_report: Get detailed member activity reports including betting patterns, deposits/withdrawals, and account status. Filterable by date range and product.
        - /get_agent_report: Get agent performance reports showing downline activity, commission earned, and member acquisition. Includes risk monitoring and compliance metrics.
        - /get_master_report: Get master agent reports with aggregated agent network data, commission structures, and risk exposure. Supports hierarchical view of network.
        - /get_super_report: Get super master reports with high-level network analytics, risk management metrics, and commission optimization insights.
        - /get_company_report: Get company-wide performance reports including P&L, turnover, active users and key business metrics across all products and user levels.
        - /get_reward_report: Get reward/bonus reports showing incentives awarded, redemption rates, and impact on user engagement. Filterable by promotion type and user segment.

    Determine which function best matches the user's request and return it in JSON format like:
    {
        "function_called": "/function_name"
    }
    """

    # Construct the prompt for function determination with examples
    user_prompt = get_user_prompt(query)

    
    format_output =  {
        "type": "object",
        "properties": {
            "function_called": {
                "type": "string",
                "description": "The name of the function to call",
                "enum": [
                    '/get_winlost_report',
                    '/get_betcount_report',
                    '/get_turnover_report',
                    '/get_net_turnover_report',
                    '/get_gross_comm_report',
                    '/get_member_report',
                    '/get_agent_report',
                    '/get_master_report',
                    '/get_super_report',
                    '/get_company_report',
                    '/get_reward_report',
                    'N/A'
                ]
            }
        },
        "required": ["function_called"]
    }

    model = 'qwen2.5:14b'
    
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
    ]

    headers = {"Content-Type": "application/json"}

    data = {
        "model": model,
        "messages": messages,
        "format": format_output,
        "stream": False
    }

    response = requests.post(url, headers=headers, data=json.dumps(data))
    return json.loads(response.json()['message']['content'])

In [16]:

# Example usage
example_inputs = queries = [
    "Get me a Win Loss Detail Report yesterday",
    "Get me a Turnover Detail Report on day 10",
    "Net Turnover Detail Report from 1/1 to 31/1",
    "Get me a Win Loss Detail Report for Direct Member who played Product Detail Sportsbook in Sportsbook Product from 01/02/2024 to 15/02/2024",
    "Give me my Win Loss Detail report  last week",
    "Give me a Gross Commission Report Detail report for Sportsbook from last week.",
    "Give me my Win Loss Detail report  last week",
    "Get me a Company Report of master1",
    "Win/Loss details for Product Sportsbook",
    "Get me a Win Loss on day 10",
    "I want Win Loss report of Sportsbook this week",
    "I want get performance of abc1 last week",
    "Live Casino Master report of Master master1 yesterday",
    "I need Member report of Agent agent01 last week",
    "Show me today's summary",
    "How did we perform in live casino last week?",
    "The amount I need to pay for master abc1 last week",
    "Get report of master1",
    "Win/Loss details for Product Sportsbook",
    "Get me a Super Report of master1"
]

for input_text in example_inputs:
    called_function = functional_calling(input_text)
    print(f"Input: {input_text}")
    print(f"Called function: {called_function['function_called']}")
    print("\n")


Input: Get me a Win Loss Detail Report yesterday
Called function: /get_winlost_report


Input: Get me a Turnover Detail Report on day 10
Called function: /get_turnover_report


Input: Net Turnover Detail Report from 1/1 to 31/1
Called function: /get_net_turnover_report


Input: Get me a Win Loss Detail Report for Direct Member who played Product Detail Sportsbook in Sportsbook Product from 01/02/2024 to 15/02/2024
Called function: /get_winlost_report


Input: Give me my Win Loss Detail report  last week
Called function: /get_winlost_report


Input: Give me a Gross Commission Report Detail report for Sportsbook from last week.
Called function: /get_gross_comm_report


Input: Give me my Win Loss Detail report  last week
Called function: /get_winlost_report


Input: Get me a Company Report of master1
Called function: /get_company_report


Input: Win/Loss details for Product Sportsbook
Called function: /get_winlost_report


Input: Get me a Win Loss on day 10
Called function: /get_winlost_r

In [2]:
functional_calling("Hello")

{'function_called': 'N/A'}

In [3]:
functional_calling("I want Sportsbook only")

{'function_called': 'N/A'}